# Quickstart Tutorial

This is a quick tutorial that will go through the bare minimum needed to set up a useful flow.

Make sure you have an Aqueduct server running locally. If you don't, run `aqueduct start` to start the server.

---

### Step 1: Set up the Aqueduct client


In [1]:
from aqueduct import Client, op, get_apikey
client = Client(get_apikey(), "localhost:8080")

/Users/kennethxu/opt/anaconda3/envs/python38/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/specifiers.py:255: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  warnings.warn(


### Step 2: Extract some data from the database
Pull some data out of the demo database, which is automatically included with the server.

In [2]:
demo_db = client.integration("aqueduct_demo")
reviews_table = demo_db.sql("select * from hotel_reviews;")

### Step 3: Perform a transformation on the data
In this case, we'll append a new column 'strlen', which contains the length of the review.

In [3]:
@op
def transform_data(reviews):
    reviews['strlen'] = reviews['review'].str.len()
    return reviews

table_with_strlen = transform_data(reviews_table)

You can materialize and inspect the resulting table using `.get()`.

In [4]:
table_with_strlen.get()

,hotel_name,review_date,reviewer_nationality,review,strlen
0,H10 Itaca,2017-08-03,Australia,Damaged bathroom shower screen sealant and ti...,82
1,De Vere Devonport House,2016-03-28,United Kingdom,No Negative The location and the hotel was ver...,84
2,Ramada Plaza Milano,2016-05-15,Kosovo,No Negative Im a frequent traveler i visited m...,292
3,Aloft London Excel,2016-11-05,Canada,Only tepid water for morning shower They said ...,368
4,The Student Hotel Amsterdam City,2016-07-31,Australia,No Negative The hotel had free gym table tenni...,167
...,...,...,...,...,...
95,The Chesterfield Mayfair,2015-08-25,Denmark,Bad Reading light And light in bathNo Positive,47
96,Hotel V Nesplein,2015-08-27,Turkey,Nothing except the construction going on the s...,456
97,Le Parisis Paris Tour Eiffel,2015-10-20,Australia,When we arrived we had to bring our own baggag...,672
98,NH Amsterdam Museum Quarter,2016-01-26,Belgium,No stairs even to go the first floor Restaura...,156


### Step 4: Save the transformed data back to the database

If you're happy with the result, you can save the data back to the database. This save is not performed until the flow is actually published.

In [5]:
table_with_strlen.save(demo_db.config(table="reviews_with_strlen", update_mode="replace")) 

### Step 5: Publish the flow

In [6]:
client.publish_flow(name="Append Strlen to Reviews", artifacts=[table_with_strlen])

Url:  http://localhost:8080/workflow/352cd2b6-7682-471d-b36d-36e632c2e05e


The published workflow is named "Append Strlen of Reviews". You can view it's runs on the UI (accessible in the browser at `localhost:8080`). It will extract from the reviews data, append a column, and write the result back into the database as a table named "reviews_with_strlen".

---

There is a lot more you can do with Aqueduct, including having flows run automatically on a cadence, parameterizing flows, and reading to and writing from many different integrations (S3, Postgres, etc.). Check out the other tutorials and examples [here](https://docs.aqueducthq.com/example-workflows) if you're interested!